<a href="https://colab.research.google.com/github/Pratye/Drone-Surveillance-Contest/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downloading Video
!wget https://www.computervision.zone/wp-content/uploads/2021/07/DRONE-SURVEILLANCE-CONTEST-VIDEO.mp4

# Installing Opencv for YoloV4 
Normal version doen not supports few activartion layers

In [ ]:
!pip uninstall opencv-python

In [ ]:
%cd /content
!git clone https://github.com/opencv/opencv
!git clone https://github.com/opencv/opencv_contrib
!mkdir /content/build
%cd /content/build
!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules /content/opencv
!make -j8 install

In [ ]:
import cv2
cv2.__version__

'4.5.3-dev'

# Main

In [ ]:
import math
import cv2 as cv

class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 1


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist<=30:
                    self.center_points[id] = (cx, cy)
                    #print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False and (300<cy<950):
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids



In [ ]:
import cv2 as cv
import numpy as np
import time
import math
from google.colab.patches import cv2_imshow
#from tracker import *

Toplam_filtered = []

# Create tracker object
tracker = EuclideanDistTracker()

# Load Yolo-Tiny ----------------------------------------------------------------------------------
net = cv2.dnn.readNet( "/content/yolov4-custom-detector.cfg", "/content/drive/MyDrive/Colab/darknet/build/darknet/x64/backup/yolov4-custom-detector_best.weights") # the yolo model

net.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
print(cv.cuda.getCudaEnabledDeviceCount())

classes = []
with open("/content/obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Loading video ------------------------------------------------------------------------
cap = cv.VideoCapture("/content/DRONE-SURVEILLANCE-CONTEST-VIDEO.mp4")
cap.set(3, 1920)  # set video widht
cap.set(4, 1080)  # set video height

font = cv.FONT_HERSHEY_COMPLEX_SMALL
starting_time = time.time()
frame_id = 0
toplam = 0
t = 0
say = 0
# Definig Output Video
video = cv2.VideoWriter('/content/video.avi', 0, 30, (1920, 1080))
# Start --------------------------------------------------------------------------------------
for q in range(1,683): #683
    # Get frame
    _, frame = cap.read()

    frame_id += 1
    if frame_id % 1 != 0:
        continue

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv.dnn.blobFromImage(frame, 0.00261, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    outs = net.forward(output_layers)

    result = []
    boxes = []
    liste = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.8:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])

    boxes_ids = tracker.update(boxes)

    # Filtering some issues -----------------------------------------------------------------------------
    eklenecekler = []
    filtered = []

    for i in range(len(boxes_ids)):
        if boxes_ids[i][4] not in eklenecekler:
            eklenecekler.append(boxes_ids[i][4])
            filtered.append(boxes_ids[i])

    for box_id in filtered:
        x, y, w, h, id = box_id

        cv.rectangle(frame, (x, y), (x + w , y + h), (0, 0, 255), 2)
        cv.rectangle(frame, (x, y), (x + w//3 +1, y + w//4), (255,0,0), -1)
        cv.putText(frame, str(id), (x, y+w//5), cv.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)
        
    Toplam_filtered = [*Toplam_filtered, *filtered]

    # Total car count -------------------------------------------------------------------------------------------------------
    try:
        _, _, _, _, value = max(filtered, key=lambda item: item[4])
        if value > toplam:
            toplam = value
    except:
        pass

    # Some texts -------------------------------------------------------------------------------------------------------------
    cv.putText(frame, "Pratye Aggarwal", (820, 90), font, 2.5, (0, 0, 255), 2)
    cv.putText(frame, str(toplam), (1700, 90), font, 3, (0, 0, 255), 3)

    if t!=toplam and toplam%12==0:
      print('==>',toplam)
      t=toplam
    elif t!=toplam:
      print('->',toplam)
      t=toplam


    # Final ------------------------------------------------------------------------------------------------------------------
    video.write(frame) 


cap.release()
video.release() 


# Convert Video.avi to mp4

In [ ]:
!pip install MoviePy

import moviepy.editor as moviepy
clip = moviepy.VideoFileClip("/content/video.avi")
clip.write_videofile("Video.mp4")